Shwetha(shwva184)
Suhani(suhar073)

# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**Before starting with this lab, make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating).** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [3]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [4]:
import spacy

def preprocess(text):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    preprocessed = nlp(text)
    return [token.lemma_ for token in preprocessed if token.is_stop == False and token.lemma_.isalpha() == True]

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [5]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer. (In scikit-learn parlance, the `preprocessor` handles string-level preprocessing.)

In [6]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=preprocess)
corpus = df['description']
X = vectorizer.fit_transform(corpus)

Test your implementation by running the following cell:

In [7]:
X.shape

(1614, 21427)

This should show the dimensions of the matrix `X` to be 1614 × 21427.

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [8]:
from sklearn.neighbors import NearestNeighbors
def search(query):
    # TODO: Replace the next line with your own code.
    q = vectorizer.transform([query])
    neig = NearestNeighbors(n_neighbors = 10, metric = "cosine")
    neig.fit(X)
    res_id = neig.kneighbors(q, return_distance=False)[0]
    return df.iloc[res_id]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [9]:
search('dodge trains')

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
192,Boxing Star,Go for the K.O.!\nMake Your Opponent See Stars...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [10]:
# TODO: Replace the next line with your own code.
idf = list(vectorizer.idf_)
#print(idf)
word = list(vectorizer.get_feature_names())
#print(word)
#type(idf)
combined_set = list(tuple(zip(idf,word)))
combined_set.sort(key = lambda x: (x[0],x[1]))

terms = [ x[1] for x in combined_set]
print(terms[:10])#lowest idf, common words

['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']


In [11]:
print(terms[-10:])#high idf , not so common words

['회원가입에', '회원을', '획득한', '효과', '효과음', 'ﬁnd', 'ﬁnger', 'ﬁnish', 'ﬁrst', 'ﬂye']


Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [12]:
print(terms[:10]) #top 10

['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']


## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [13]:
import numpy as np 
text = df['description'][1428]
#sort_text_ids = np.array(vectorizer.transform([text]).todense())
#ids = np.argsort(sort_text_ids)

words = vectorizer.get_feature_names()
ids= np.flip(np.array(np.argsort(vectorizer.transform([text])[0].todense(),axis=None)))
[words[i] for i in ids[0][0:10]]


#np.argsort(text, axis=- 1, kind=None, order=None)





['train',
 'railway',
 'railroad',
 'rail',
 'chaos',
 'crash',
 'timetable',
 'haul',
 'overcast',
 'locomotive']

In [14]:
def keywords(text, n=10):
    # TODO: Replace the next line with your own code.
    ids= np.flip(np.array(np.argsort(vectorizer.transform([text])[0].todense(),axis=None)))
    words = vectorizer.get_feature_names()
    
    
    return [words[i] for i in ids[0][0:n]]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [15]:
print(keywords(df['description'][1428]))

['train', 'railway', 'railroad', 'rail', 'chaos', 'crash', 'timetable', 'haul', 'overcast', 'locomotive']


This should give the following output:

## Reflection questions

The following reflection questions are questions that you could be asked in the oral exam. Try to answer each of them in the form of a short text and put it in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *domain-specific* or *application-specific* stop words?

**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

Answer 1.1 : Stop words are removed because they are non informative and they simply add to overhead. Lemmatising the text converrts the text to its root form , this helps us to get the correct count of how many times the particular word is used, but having different forms of this word will hinder with this as we will end up considering different forms of the words as different words.
example for application specific stop words : if an application is working on finding out the most used ingredients in skincare products for a particular skin type or skin concern by going through the ingredient list of all the skin care products , the information such as % of a ingredient mention in the list might not be useful, so in this casse we can use integer% as a stop word and ignore it.

Answer 1.2 : Dimention of X corresponds to Tf-idf-weighted document-term matrix ie it is a sparse matrix of (nsamples , nfeatures). Here rows represent each document and columns represent the vocabulary in each document. This is in row compressed format , ie only the non zero terms are represented.

Answer 1.3 : Lower idf score means there is high occurance of the word in the document and higher idf means the word is not very common in the document. Stop words occurs many times in our document and they need to be systematically  discounted , this is idf part of tf-idf. More frequently a word appears it becomes less significant to differentiate  the given document. Using Tf-idf helps us find distinct and frequent words as markers
reference : https://wiki.pathmind.com/bagofwords-tf-idf

**Congratulations on finishing L1! 👍**